In [ ]:
import sys
!{sys.executable} -m pip install transformers
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install torchvision

In [11]:

from transformers import AutoTokenizer, AutoModel
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from torchvision import datasets
import torch
import torch.nn as nn
import torch.optim as optim
import csv



def gettokennumber(sentence, number):
    inputs = tokenizer(sentence)
    word=0
    token=0
    hyphen=0
    for tokenid in inputs["input_ids"]:
        if str(tokenizer.decode(tokenid))=="-":
            hyphen=hyphen+1
        if str(tokenizer.decode(tokenid)).startswith("#")==False:
            if word==number+2+hyphen:
                return token-1
            word=word+1   
        token=token+1
##needs fixing
def preparedata(data):
    RetVal=[]
    for sentence, number, _ in data:
        tokennumber=gettokennumber(sentence, number)
        inputs=tokenizer(sentence, return_tensors="pt")
        RetVal.append(bertmodel(**inputs)[0][0][tokennumber])
    return RetVal
        

def casetonumber(case):
    switch = {
        "Nom": 0,
        "Acc": 1,
        "Ins": 2,
        "Ine": 3, 
        "Sup": 4,
        "Sub": 5
    }
    return switch.get(case, "Invalid case")



In [2]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
bertmodel = AutoModel.from_pretrained("bert-base-multilingual-cased")

file1 = open("train.tsv")
train_tsv=csv.reader( file1, delimiter="\t")

file2 = open("test.tsv")
test_tsv=csv.reader( file2, delimiter="\t")

training_data=[]
test_data=[]


i=0 
for obj in train_tsv:
    training_data.append([])
    training_data[i].append(obj[0])
    training_data[i].append(obj[2])
    training_data[i].append(casetonumber(obj[3]))
    i=i+1

i=0
for obj in test_tsv:
    test_data.append([])
    test_data[i].append(obj[0])
    test_data[i].append(obj[2])
    test_data[i].append(casetonumber(obj[3]))
    i=i+1

trainloader=torch.utils.data.DataLoader(training_data, batch_size=128, shuffle=True)
testloader=torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=True)
file1.close()
file2.close()


DONE


In [12]:

class SimpleClassifier(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim):
        super().__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, X):
        h = self.input_layer(X)
        h = self.relu(h)
        out = self.output_layer(h)
        return out

In [37]:
print(output[0][0][3].size())

torch.Size([768])


In [13]:
Net = SimpleClassifier(
    input_dim=768,
    output_dim=6,
    hidden_dim=50
)
Net

SimpleClassifier(
  (input_layer): Linear(in_features=768, out_features=50, bias=True)
  (relu): ReLU()
  (output_layer): Linear(in_features=50, out_features=6, bias=True)
)

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Net.parameters())

In [17]:
def train(epoch):
    Net.train()
    running_loss=0.0
    correct = 0.0
    total=0
    for i, data in enumerate(trainloader, 0):
        _, _, labels = data
        inputs = preparedata(data)
        optimizer.zero_grad()
        outputs=Net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            running_loss +=loss.items()
            _, predicted = torch.max(outputs, 1)
            correct += predicted.eq(label).sum().item()
            total += label.size(0)
            
            
    tr_loss = running_loss/i
    tr_corr = correct/total*100
    print("Train epoch %d correct: %.2f%" %(epoch + 1, tr_corr))
    return tr_loss, tr_corr

def val(epoch):
    Net.eval()
    running_loss=0.0
    correct = 0.0
    total=0
    for i, data in enumerate(testloader, 0):
        _, _, labels = data
        inputs = preparedata(data)

        with torch.no_grad():
            outputs=Net(inputs)
            loss = criterion(outputs, labels)
            running_loss +=loss.items()
            _, predicted = torch.max(outputs, 1)
            correct += predicted.eq(label).sum().item()
            total += label.size(0)
            
            
    val_loss = running_loss/i
    val_corr = correct/total*100
    print("Test epoch %d correct: %.2f%" %(epoch + 1, tr_corr))
    return val_loss, val_corr

In [18]:

train_accs = []
train_losses = []
val_accs = []
val_losses = []
best_acc = 0
torch.manual_seed(42)
numEpoch=10
for epoch in range(numEpoch):
    #Train
    loss, acc = train(epoch)
    train_accs.append(acc)
    train_losses.append(loss)
    #val
    loss, acc = val(epoch)
    val_accs.append(acc)
    val_losses.append(loss)
    scheduler.step()
    if acc>best_acc:
        best_acc = acc
        print("Best model so far")
        torch.save(net, "model.pth")

ValueError: too many values to unpack (expected 3)